In [109]:
from langchain_chroma import Chroma
import os
from pyprojroot import here
#from langchain_openai import OpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings
from openai import OpenAI
from openai import AzureOpenAI
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

**Load environment variables and configs**

In [110]:
os.environ['AZURE_OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")

EMBEDDING_MODEL = "text-embedding-3-small"
VECTORDB_DIR = "data/airline_policy_vectordb"
K=2

**Load the vectorDB**

In [111]:
vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=AzureOpenAIEmbeddings(model=EMBEDDING_MODEL)
)
print("Number of vectors in vectordb:",
      vectordb._collection.count(), "\n\n")

Number of vectors in vectordb: 0 




**Sample Query**

In [112]:
message = "What is the cancelation rule for a flight ticket at swiss airline policy?"

**Perform the vector Search**

In [113]:
docs = vectordb.similarity_search(message, k=K)

In [114]:
docs

[]

**Prepare the prompt for the GPT model**

In [115]:
question = "# User new question:\n" + message
retrieved_content = ""
for doc in docs:
    retrieved_content += f"{doc.page_content}\n\n"
prompt = f"# Content:\n{retrieved_content}\n\n{question}"

Prepared prompt

In [116]:
pprint(prompt)

('# Content:\n'
 '\n'
 '\n'
 '# User new question:\n'
 'What is the cancelation rule for a flight ticket at swiss airline policy?')


**Pass the prompt to the GPT model and get the response**

In [117]:
client = AzureOpenAI(
    api_key="EnvrvVgQgVNjzHrXY57TUyESTNeIqovna3WsB7mctCSmyrICEJ2lJQQJ99AJACHYHv6XJ3w3AAABACOG2fTJ",
    azure_deployment="gpt4omini",
    azure_endpoint ="https://aressgenaisvc.openai.azure.com/",
    api_version="2024-06-01"
)
response = client.chat.completions.create(
     model= "gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You will receive a user's query and possible content where the answer might be. If the answer is found, provide it, if not, state that the answer does not exist."},
        {"role": "user", "content": prompt}
    ]
)

Printing the response

In [118]:
pprint(response.choices[0].message.content)

'The answer does not exist.'


**RAG Tool design using LangChain**

In [119]:
from langchain_core.tools import tool

@tool
def lookup_swiss_airline_policy(query: str)->str:
    """Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query."""
    vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=AzureOpenAIEmbeddings(model=EMBEDDING_MODEL)
    )
    docs = vectordb.similarity_search(query, k=K)
    return "\n\n".join([doc.page_content for doc in docs])

In [120]:
print(lookup_swiss_airline_policy.name)
print(lookup_swiss_airline_policy.args)
print(lookup_swiss_airline_policy.description)

lookup_swiss_airline_policy
{'query': {'title': 'Query', 'type': 'string'}}
Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query.


In [121]:
print(lookup_swiss_airline_policy.invoke("can I cancel my ticket?"))